In [9]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the model from the H5 file
h5_model_path = 'YamNet_Transfer_Learning.h5'
model = load_model(h5_model_path)

# Define the path to save the TensorFlow SavedModel
saved_model_path = 'testing_model_yamnet'

# Save the model in SavedModel format
tf.saved_model.save(model, saved_model_path)

print(f"Model saved in SavedModel format at: {saved_model_path}")


INFO:tensorflow:Assets written to: testing_model_yamnet\assets
Model saved in SavedModel format at: testing_model_yamnet


In [10]:
import tensorflow as tf
import tensorflow_hub as hub

# Define the custom ReduceMeanLayer
class ReduceMeanLayer(tf.keras.layers.Layer):
    def __init__(self, axis=0, **kwargs):
        super(ReduceMeanLayer, self).__init__(**kwargs)
        self.axis = axis

    def call(self, inputs):
        return tf.reduce_mean(inputs, axis=self.axis)

# Path to the SavedModel
saved_model_path = './testing_model_yamnet'

# Load the SavedModel using the TensorFlow SavedModel API
loaded_model = tf.saved_model.load(saved_model_path)

# Get the input and output signatures from the loaded model
infer = loaded_model.signatures["serving_default"]

# Use the correct input key
input_key = 'input_1'
input_tensor = infer.structured_input_signature[1][input_key]

# Create a new input layer that matches the expected input shape and type
input_segment = tf.keras.layers.Input(shape=input_tensor.shape[1:], dtype=input_tensor.dtype, name='audio')

# Use the existing model to get the embeddings
yamnet_embedding_extractor = hub.KerasLayer(saved_model_path, trainable=False)
embeddings_output = yamnet_embedding_extractor(input_segment)

# Apply the custom ReduceMeanLayer
serving_outputs = ReduceMeanLayer(axis=0, name='classifier')(embeddings_output)

# Create the new model
serving_model = tf.keras.Model(inputs=input_segment, outputs=serving_outputs)

# Save the new model in SavedModel format
new_saved_model_path = './new_fused_model'
serving_model.save(new_saved_model_path, include_optimizer=False)

print(f"New model saved in SavedModel format at: {new_saved_model_path}")


INFO:tensorflow:Assets written to: ./new_fused_model\assets
New model saved in SavedModel format at: ./new_fused_model
